In [1]:
import numpy as np
import nltk
import string
import requests
import re
import glob
import os, sys
import datetime
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import xlrd
import csv
import time
import dask
import dask.dataframe as dd
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from sklearn.feature_extraction.text import TfidfVectorizer
import shutil
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
import nltk
from data_preparation import clean_ticker, download_to_item1
nltk.download('stopwords')
dl = Downloader("/home/hongzhuoqiao/10K_Projects/sec_filings")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hongzhuoqiao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
company_df = pd.read_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/multiple_observations.txt", sep='\t')
tc_df = company_df[['cusip','tic']].drop_duplicates()
tc_df['ticker'] = tc_df['tic'].apply(clean_ticker)
tc_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tc.csv")
tc_df.head()

,cusip,tic,ticker
0,000361105,AIR,AIR
13,000886309,ADCT.1,ADCT
17,001058205,IWKS,IWKS
19,020813101,ALO.2,ALO
20,02376R102,AAL,AAL


In [29]:
# 2nd time integration of all result_df
# no need to run this cell
df_list =[]
for i in range(1,45):
    df_list.append(pd.read_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/result_csv/result_"+str(i)+".csv", index_col=0))

df_list.append(pd.read_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/result_csv/result_ARCW.csv", index_col=0))
df_list.append(pd.read_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/result_csv/result_nofile_havefile.csv", index_col=0))
result_df = pd.concat(df_list)
result_df.drop_duplicates(subset='cusip', keep='last', inplace=True)
result_df['extraction_info'] = result_df['extraction_info'].apply(lambda x: x[1:-1])
result_df['totals'] = result_df['extraction_info'].str.split(",", expand=True)[0].astype(int)
result_df['extracted'] = result_df['extraction_info'].str.split(",", expand=True)[1].astype(int)
result_df['errors'] = result_df['extraction_info'].str.split(",", expand=True)[2].astype(int)
count = len(result_df[result_df['totals']!=0])
count_0 = len(result_df[result_df['totals']==0])
print("{} Companies have 10k files extracted from the API.".format(count))
print("{} Companies have 10k files cannot be extracted from the API.".format(count_0))
#result_df.to_csv("result.csv")
result_df.head()

7842 Companies have 10k files extracted from the API.
11983 Companies have 10k files cannot be extracted from the API.


,cusip,tic,ticker,extraction_info,totals,extracted,errors
0,000361105,AIR,AIR,"26, 26, 0, []",26,26,0
13,000886309,ADCT.1,ADCT,"18, 18, 0, []",18,18,0
17,001058205,IWKS,IWKS,"0, 0, 0, []",0,0,0
19,020813101,ALO.2,ALO,"14, 14, 0, []",14,14,0
20,02376R102,AAL,AAL,"27, 27, 2, ['08', '09']",27,27,2


In [7]:
# no need to re-run
nofile_df = result_df[result_df['totals']==0]
nofile_df['ticker'] = nofile_df['ticker'].apply(clean_ticker)
nofile_df['totals_2'] = nofile_df['ticker'].apply(lambda x: dl.get("10-K", x))
nofile_havefile_df = nofile_df[nofile_df['totals_2']!=0]
nofile_havefile_df.to_csv('nofile_havefile.csv')
nofile_havefile_df.head(30)

,cusip,tic,ticker,extraction_info,totals,extracted,errors
17,001058205,IWKS,IWKS,"0, 0, 0, []",0,0,0
44,G3156P103,ASA,ASA,"0, 0, 0, []",0,0,0
146,617577101,MRCBF,MRCBF,"0, 0, 0, []",0,0,0
158,004616207,ACMTA,ACMTA,"0, 0, 0, []",0,0,0
182,022167100,AHIX,AHIX,"0, 0, 0, []",0,0,0


In [19]:
# no need to re-run 
import import_ipynb
import _10k_multi_company_item1_extraction as extraction
#nofile_havefile_df = pd.read_csv('nofile_havefile.csv', index_col=0)
nofile_havefile_df['extraction_info'] = nofile_havefile_df['ticker'].apply(lambda x: extraction.download_to_item1(x))
#nofile_havefile_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/result_nofile_havefile.csv")
nofile_havefile_df.head(30)
#nofile_havefile_df[['cusip','tic','ticker','extraction_info']].to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/result_nofile_havefile.csv")
# go back to re merge the new result_df

 organic growth and acquisitions of other banks, including failed, underperforming and undercapitalized banks. We have raised $955.6 million to make acquisitions through a series of private placements and an initial public offering of our common stock. Since inception, we have acquired seven depository institutions, including the assets and certain deposits of three failed banks from the Federal Deposit Insurance Corporation (“FDIC”). We recently announced our eighth acquisition, CommunityOne Bancorp (“CommunityOne”) which we expect will close during the second calendar quarter of 2016, subsequent to the completion of required regulatory examinations and subject to the approval of shareholders and regulators. We currently operate 152 branches in Florida, North and South Carolina, Tennessee, and 
>>> New item1 new file for year CBF-16 is saved

CBF's folder has been deleted!
>>> Got 1 for PVBC
>>> Created new folder for PVBC
>>> Start extracting item1 sections for PVBC
 

>>>Starting op

,cusip,tic,ticker,extraction_info,totals,extracted,errors,totals_2
420,87509Q105,ACEL.,ACEL,"(3, 3, 0, [])",0,0,0,3
931,030137103,AMK.2,AMK,"(1, 1, 0, [])",0,0,0,1
5523,211919105,CNVR.,CNVR,"(14, 14, 0, [])",0,0,0,14
7949,G47791101,IR,IR,"(3, 3, 0, [])",0,0,0,3
9706,577723109,MAXC.,MAXC,"(11, 11, 1, [06])",0,0,0,11
9721,577903107,MAXE.,MAXE,"(13, 13, 0, [])",0,0,0,13
15041,885535104,COMS.,COMS,"(16, 16, 0, [])",0,0,0,16
16000,835637109,SWHI.,SWHI,"(16, 16, 5, [06, 05, 09, 95, 07])",0,0,0,16
18730,09238D205,BHR.,BHR,"(7, 7, 0, [])",0,0,0,7
20157,844909200,SWTX.1,SWTX,"(1, 1, 0, [])",0,0,0,1


In [14]:
# Clean exsiting file section
# Check if company's file need to be modified

#TODO: find better solution to clean
ticker = 'A'
path_header = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section"
path_for_company_folder = path_header + "/" + ticker +"/*.txt"
files = glob.glob(path_for_company_folder)
file_list =[]
filename_list =[]
for file in files:
    f = open(file, 'r')
    print('\n')
    print(os.path.basename(file))
    print(f.read()[:350])



A-15.txt
>Item 1.    Business OverviewAgilent Technologies Inc. ("we", "Agilent" or the "company"), incorporated in Delaware in May 1999, is a global leader in life sciences, diagnostics and applied chemical markets, providing application focused solutions that includes instruments, software, services and consumables for the entire laboratory workflow.On No


A-16.txt
>Item 1.    Business OverviewAgilent Technologies Inc. ("we", "Agilent" or the "company"), incorporated in Delaware in May 1999, is a global leader in life sciences, diagnostics and applied chemical markets, providing application focused solutions that includes instruments, software, services and consumables for the entire laboratory workflow.On No


A-04.txt
>Item 1
  

Business
  
3

 


A-02.txt
>Item 1
    
    
     

Business
    
 
 
2
 



>Item 1.     Business

Overview

     Business
     
Agilent Technologies, incorporated in Delaware in
May 1999, is a global diversified technology company that
provides enabl

In [ ]:
# below are backup cells

In [328]:
ticker = 'AAPL'
# Test on Single company
doc_test_list =['  <Hello% my_World 2020! BYE    2019@!!   ', 'item.1A HELLO the WORLD 2019.']

# Preprocessing
def preprocess(docs):
    sepcial_stopwords =['item','items','itemsa'] # create sepcial stopwords set
    docs = [doc.lower() for doc in docs] # lowercase
    table = str.maketrans('','', string.punctuation) # remove punctuation 1
    docs = [doc.translate(table) for doc in docs] # remove punctuation 2
    docs = [re.sub(r'\d+', 'num', doc) for doc in docs] # replace all # to 'num'
    docs = [re.sub('[^\w\s]', '',doc) for doc in docs] # remove special char
    docs = [re.sub('_', '', doc) for doc in docs] # remove underscore
    docs = [re.sub('\s+',' ', doc) for doc in docs] # change whitespaces to one space
    stopwords = set(nltk.corpus.stopwords.words('english')+sepcial_stopwords) # create stopwords set with special stopword if have any 1
    docs = [[word for word in doc.split() if word not in stopwords] for doc in docs]  # create stopwords set with special stopword if have any 2
    stemmer = nltk.stem.PorterStemmer() # stemming 1
    docs = [" ".join([stemmer.stem(word) for word in doc]) for doc in docs] # stemming 2
    docs = [doc.strip() for doc in docs]
    return docs

def similarity_calculation(tikcer):
    path_header = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section"
    path_for_company_folder = path_header + "/" + ticker +"/*.txt"
    files = glob.glob(path_for_company_folder)
    file_list =[]
    filename_list =[]
    for file in files:
        f = open(file, 'r')
        filename = os.path.basename(file)
        file_list.append(f.read())
        filename_list.append(filename[-6:-4])

    filename_list = [datetime.datetime.strptime(f,'%y').strftime('%Y') for f in filename_list]
    file_list = preprocess(file_list)
    tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii').fit_transform(file_list)
    pairwise_similarity = tfidf*tfidf.T
    sim_df = pd.DataFrame(pairwise_similarity.toarray(), columns = filename_list, index = filename_list)
    sim_df.sort_index(axis=0, ascending=False, inplace=True)
    sim_df.sort_index(axis=1, ascending=False, inplace=True)
    return sim_df
    last_year_sim_list =[]
    # for i in range(0,len(sim_df)-1):
    #     last_year_sim_list.append(sim_df.iloc[i,i+1])
    # last_year_sim_df = pd.DataFrame(last_year_sim_list, index= sim_df.columns[:-1], columns=[ticker])
    # return last_year_sim_df

#similarity_calculation(ticker).to_csv('apple_sim_example.csv')
similarity_calculation(ticker)

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994
2019,1.000000,0.976045,0.964569,0.957833,0.949010,0.937839,0.939326,0.949884,0.950251,0.930780,...,0.619571,0.631360,0.674468,0.661353,0.717798,0.744946,0.710893,0.711195,0.713230,0.697029
2018,0.976045,1.000000,0.989524,0.985745,0.979088,0.969009,0.965515,0.963590,0.962885,0.941606,...,0.664119,0.671468,0.713306,0.693330,0.736832,0.765894,0.732724,0.736589,0.733175,0.717382
2017,0.964569,0.989524,1.000000,0.992558,0.987448,0.980058,0.975114,0.970736,0.966113,0.944624,...,0.687406,0.693997,0.734854,0.715103,0.757282,0.783111,0.751202,0.753964,0.747205,0.729627
2016,0.957833,0.985745,0.992558,1.000000,0.992068,0.984501,0.980709,0.975971,0.971050,0.947506,...,0.682775,0.687832,0.731327,0.707998,0.748494,0.776989,0.744061,0.748094,0.742109,0.725097
2015,0.949010,0.979088,0.987448,0.992068,1.000000,0.991011,0.983383,0.978439,0.969907,0.952122,...,0.697879,0.705872,0.750140,0.728372,0.765022,0.792956,0.759583,0.764251,0.755737,0.737694
2014,0.937839,0.969009,0.980058,0.984501,0.991011,1.000000,0.993275,0.980517,0.970789,0.953211,...,0.707468,0.708822,0.748650,0.725167,0.762149,0.785745,0.751015,0.753234,0.746178,0.726550
2013,0.939326,0.965515,0.975114,0.980709,0.983383,0.993275,1.000000,0.986325,0.974002,0.955035,...,0.693423,0.690183,0.729086,0.703306,0.745422,0.768025,0.731543,0.731798,0.730884,0.711756
2012,0.949884,0.963590,0.970736,0.975971,0.978439,0.980517,0.986325,1.000000,0.986523,0.969691,...,0.700018,0.696110,0.734201,0.703961,0.749258,0.771879,0.733158,0.737513,0.737370,0.717821
2011,0.950251,0.962885,0.966113,0.971050,0.969907,0.970789,0.974002,0.986523,1.000000,0.978898,...,0.700998,0.701795,0.743000,0.715943,0.762592,0.784422,0.748280,0.748789,0.749365,0.726391
2010,0.930780,0.941606,0.944624,0.947506,0.952122,0.953211,0.955035,0.969691,0.978898,1.000000,...,0.714283,0.713918,0.754339,0.729333,0.771657,0.791734,0.755050,0.755163,0.754114,0.730143
